### The problem of long-term dependencies
* RNNs connect previous information to present task:
> enough for predicting the next word for "the clouds are in the **sky**"

> may not be enough when more context is needed: "I grew up in France... I speak fluent **French**"

### RNN
* All recurrent neural networks have the form of a chain of repeating modules of neural network
> ht = tanh(W\[ht-1, xt-1\])

### LSTM
* LSTMs also have this chain like structure, but the repeating module has a different structure. Instead of having a single neural network layer there are four, interacting in a very special way.
* The core idea behind LSTMs: **Cell State**
> Gates are a way to optionally let information through. They are composed out of a sigmoid neural net layer and a pointwise multiplication operation. An LSTM has three of these gates, to protect and control the cell state

#### LSTM: Forget gate
> ft = sigmoid(Wf\[ht-1, xt\] + bf)
>> It looks at ht-1 and xt and outputs a number between 0 and 1 for each number in the cell state Ct-1.
>> A 1 represents "completely keep this" while a 0 represents "completely get rid of this"

#### LSTM: Input gate and Cell state
* The next step is to decide what new information we're going to store in the cell state
> a sigmoid layer called the "**input gate layer** decides which values we'll update."
>> it = sigmoid(Wi\[ht-1, xt\] + bi)

> a tanh layer creates a vector of new candidate values, that could be added to the state.
>> Ct^ = tanh(WC\[ht-1, xt\] + bC)

* It's now time to update the old cell state into the new cell state
> Ct = ft * Ct-1 + it * Ct^
>> We multiply the old state by ft forgatting the things we decided to forget earlier. Then, we add the new candidate values, scaled by how much we decided to update each state value.

#### LSTM: Output
* Finally, we need to decide what we're going to output.
> First, we run a sigmoid layer which decides what parts of the cell state we're going to output.
>> ot = sigmoid(Wo\[ht-1, xt\] + bo)

> Then, we put the cell state through tanh (to push the values to be between -1 and 1) and multiply it by the output of sigmoid gate, so that we only output the parts we decided to.
>> ht = ot * tanh(Ct)

#### Intuitive Pipeline
* LSTM memory Cell
> Forget irrelevant parts of previous state --> Selectively update cell state values --> Output certain parts of cell state

* input gate : forget gate : behavior 
> o : 1 : remember the previous value, 1 : 1 : add to the previous value, 0 : 0 : erase the value, 1 : 0 : overwrite the value